# Configuring Aruba Network Automation Solutions - Lab4.2 Pyaoscx Workflow
## Note:1.Before starting this lab, please clear all the output (Select Kernel, then click "Restart Clear output") 2.Ensure you did not ignore running any code cell.

### There are two Approaches to use pyaoscx: open granulated approach and Imperative Factory Approach to utilize Pyaoscx library.

## Requirements:
#### On Access switch:
-	Utilize open granulated approach to create vlan200, add a description and put Interface into the VLAN. 
-	Utilize Imperative Factory Approach to create VLAN201.
-	Chanlenges: understand how to handle configurarion. Save the running-config to checkpoint and TFTP server.


In [ ]:
#Installing library needs some time. wait to see the installation result before you execute next cell. Be patient!
!pip install pyaoscx 

In [ ]:
#get to know the pyaoscx and where it is installed. wait to see the output before you execute next cell. 
!pip show pyaoscx

In [ ]:
# Import modules.
# You won't see any outputs.
from pyaoscx.session import Session
from pyaoscx.pyaoscx_factory import PyaoscxFactory
from pyaoscx.vlan import Vlan
from pyaoscx.interface import Interface
from pyaoscx.static_route import StaticRoute
from pyaoscx.vrf import Vrf
import urllib3
urllib3.disable_warnings()

In [ ]:
# There are two approaches to workflows, both using the session.
# For the version, we have two options "1" and "10.04". In this lab we choose "10.04".
# Replace <Your Access switch Management IP> with your Access switch IP(10.251.X.103)
# Replace <Your Switch Username> and <Your Switch Password> with your credential: admin/aruba123.
# You won't see any outputs.Go ahead to execute next code cell.

version = '10.04'
switch_ip = '<Your Access switch Management IP> '
s = Session(switch_ip, version)
s.open('<Your Switch Username>', '<Your Switch Password>')


#  APPROACH 1: OPEN GRANULATED APPROACH
#### 

In [ ]:
# Create Vlan object -- Not yet materialized
# Vlan is a Python Class that has been defined in PYTHON module pyaoscxl.vlan
# vlan200 is a object or instance of this Class
# You won't see any outputs.

vlan200 = Vlan(s, 200, name="VLAN 200", voice=True)


In [ ]:
# Since object is not materialized, performs a POST request -- This method internally makes a GET request right after the POST
# Obtaining all attributes VLAN related
# If you get an "Internal server error", that means the vlan is already exsisted.
# You are supposed to see "True" which means the vlan is created.

vlan200.apply()


In [ ]:
# Display all Vlans with below code. You can also go to CLI to check if the vlan has been created.

Vlan.get_all(s)


In [ ]:
#Add description for vlan200
#Print function will put "vlan200.desacrition" into the "{}" then print out the result.

vlan200.description = "New description, changed via pyaoscx SDK"
vlan200.apply()

# Now vlan200 contains the description attribute

print("VLAN 200 description {}".format(vlan200.description))


In [ ]:
# Now create another object, that we know already exists inside of the Switch
# Notice that you didn't "apply" the vlan1 because the vlan1 has already existed
vlan1 = Vlan(s, 1)


# a GET request is called to obtain the VLAN's information.
# The information is then added to the object as attributes.

vlan1.get()


In [ ]:
# Now, you are able to modify the objects internal attributes

vlan1.voice = True

# Apply changes
changed = vlan1.apply()

# If changed is True, a PUT request was done and object was modified.
# You won't see any outputs. You can go to CLI to check if the voice vlan has been applied to vlan1.


#  APPROACH 2: IMPERATIVE FACTORY APPROACH

## pyaoscx.pyaoscx_factory provide a class named PyaoscxFactoty to instantiate all pyaoscx Modules through specific methods. https://github.com/aruba/pyaoscx/blob/master/pyaoscx/pyaoscx_factory.py

### 

In [ ]:
# Create VLAN 201
# Create Factory object, passing the Session Object

factory = PyaoscxFactory(s)


In [ ]:
# Create Vlan object
# If vlan is non-existent, Factory instantly creates it inside the switch device

vlan201 = factory.vlan(201, "NAME201")


In [ ]:
# Display all Vlans with below code. You can also go to CLI to check if the vlan has been created.

Vlan.get_all(s)


In [ ]:
# At the end, the session MUST be closed

s.close()

# Challenges:
### •	1. Get the Access switch running-config configuration
### •	2. Save it to a checkpoint named “checkpoint1_by_pyaoscx”
### •	3. Backup running-config to tftp server 10.251.X.90 (X is your table number)


#=======================================================================================

# Solution:

### Note:You have to change the below cell to "code" type and move them to the above the "s.close" cell to run.
 
#### # Replace TFTP_Server_IP with your wired VLT(10.251.X.90)
    config1 is an object of Class Configuration. "get_full_config" , "create_checkpoint" and "backup_configuration" are functions under the class. 

from pyaoscx.configuration import Configuration 

config1=Configuration(s)

Configuration.get_full_config(config1,"startup-config")

Configuration.create_checkpoint(config1,"running-config","checkpoint1_by_pyaoscx11")

Configuration.backup_configuration(config1, "running-config", output_file="test_config",
                             vrf="<VRF name", config_type='json',
                             remote_file_tftp_path="tftp://<TFTP server IP>/test_config")  
                             


#  You completed the LAB4.2!
#### 